In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audio-retrieval/audio_retrieval/test.py
/kaggle/input/audio-retrieval/audio_retrieval/multilingual_retrieval_system.py
/kaggle/input/audio-retrieval/audio_retrieval/interactive_query.py
/kaggle/input/audio-retrieval/audio_retrieval/xmultilingual_retrieval_system.py
/kaggle/input/audio-retrieval/audio_retrieval/unified_index/audio_index.faiss
/kaggle/input/audio-retrieval/audio_retrieval/unified_index/metadata.json
/kaggle/input/audio-retrieval/audio_retrieval/unified_index/statistics.json
/kaggle/input/audio-retrieval/audio_retrieval/unified_index/text_index.faiss
/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/metadata/dataset_metadata.json
/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/audio_features/clap_embeddings.npy
/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/text_features/clap_text_embeddings.npy
/kaggle/input/audio-retrieval/audio_retrieval/venv/.gitignore
/kaggle/input/audio-retrieval/audio_retrieva

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.0 MB/s eta 0:00:00:00:0100:01


In [8]:
pip install transformers torch torchaudio 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [12]:
from transformers import ClapModel, ClapProcessor
# Load CLAP (HuggingFace HF version)
device = "cuda" if torch.cuda.is_available() else "cpu"

clap_processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
clap_model = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)
clap_model.eval()

ClapModel(
  (text_model): ClapTextModel(
    (embeddings): ClapTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ClapTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x ClapTextLayer(
          (attention): ClapTextAttention(
            (self): ClapTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ClapTextSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [21]:
"""
Multilingual Audio Retrieval System
Supports English (LibriSpeech) and Tamil (Indic TTS)
Uses CLAP embeddings for both audio and text
"""

import numpy as np
import json
import faiss
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# multilingual_retrieval.py
"""
Multilingual Audio Retrieval System
Supports English (LibriSpeech) and Tamil (Indic TTS)
Uses CLAP embeddings for both audio and text
"""

import json
from pathlib import Path
from typing import List, Dict, Optional, Any, Tuple
import numpy as np
import faiss

# NOTE: heavy imports (transformers / torch) are done lazily inside CLAPEncoder


class CLAPEncoder:
    """
    Lazy CLAP wrapper. It only imports transformers/torch when first used.
    encode_texts(list[str]) -> np.ndarray (N, D) float32, L2-normalized
    """
    def __init__(self, model_name: str = "laion/clap-htsat-fused", device: Optional[str] = None):
        self.model_name = model_name
        self.device = device
        self._loaded = False
        self.processor = None
        self.model = None

    def _ensure_loaded(self):
        if self._loaded:
            return
        try:
            from transformers import AutoProcessor, ClapModel
        except Exception as e:
            raise RuntimeError("transformers is required for CLAP. Install with `pip install transformers`") from e

        self.processor = AutoProcessor.from_pretrained(self.model_name)
        self.model = ClapModel.from_pretrained(self.model_name)

        # Move to device if torch found
        try:
            import torch
            dev = self.device or ("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(dev)
        except Exception:
            # continue on CPU if torch not available or move fails
            pass

        try:
            self.model.eval()
        except Exception:
            pass

        self._loaded = True

    def encode_texts(self, texts: List[str], batch_size: int = 16) -> np.ndarray:
        if not texts:
            return np.zeros((0, 0), dtype="float32")

        self._ensure_loaded()
        import torch  # require torch for tensor ops

        all_embs = []
        device = self.model.device if hasattr(self.model, "device") else (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            proc = self.processor(text=batch, return_tensors="pt", padding=True)
            proc = {k: v.to(device) for k, v in proc.items()}
            # Many ClapModel variants expose get_text_features
            try:
                with torch.no_grad():
                    out = self.model.get_text_features(**proc)
                emb = out.detach().cpu().numpy()
            except AttributeError:
                # fallback to calling model and extracting plausible outputs
                with torch.no_grad():
                    out = self.model(**proc)
                if isinstance(out, dict):
                    for key in ("text_embeds", "text_features", "pooler_output", "last_hidden_state"):
                        if key in out and out[key] is not None:
                            t = out[key]
                            emb = t.detach().cpu().numpy() if hasattr(t, "detach") else np.asarray(t)
                            break
                    else:
                        raise RuntimeError("Could not extract text features from model output.")
                else:
                    # might be a tensor
                    if hasattr(out, "detach"):
                        emb = out.detach().cpu().numpy()
                    else:
                        raise RuntimeError("Unexpected model output while encoding text.")
            all_embs.append(np.asarray(emb))

        embs = np.vstack(all_embs).astype("float32")
        # Defensive L2 normalization
        norms = np.linalg.norm(embs, axis=1, keepdims=True)
        norms[norms == 0] = 1.0
        embs = embs / norms
        return embs


class MultilingualAudioRetrieval:
    def __init__(self, clap_encoder: Optional[Any] = None):
        self.english_data = {}
        self.tamil_data = {}
        self.combined_audio_index = None
        self.combined_text_index = None
        self.combined_metadata: List[Dict] = []
        self.clap_encoder = clap_encoder

    def set_clap_encoder(self, encoder: Any):
        self.clap_encoder = encoder

    # --- data loading methods (same as your original) ---
    def load_english_data(self, audio_emb_path: str, text_emb_path: str, metadata_path: str) -> bool:
        try:
            self.english_data['audio_embeddings'] = np.load(audio_emb_path)
            self.english_data['text_embeddings'] = np.load(text_emb_path)
            with open(metadata_path, 'r', encoding='utf-8') as f:
                self.english_data['metadata'] = json.load(f)
            for item in self.english_data['metadata']:
                item['language'] = 'english'
            return True
        except Exception as e:
            print("Error loading English:", e)
            return False

    def load_tamil_data(self, audio_emb_path: str, text_emb_path: str, metadata_path: str) -> bool:
        try:
            self.tamil_data['audio_embeddings'] = np.load(audio_emb_path)
            self.tamil_data['text_embeddings'] = np.load(text_emb_path)
            with open(metadata_path, 'r', encoding='utf-8') as f:
                self.tamil_data['metadata'] = json.load(f)
            for item in self.tamil_data['metadata']:
                item['language'] = 'tamil'
            return True
        except Exception as e:
            print("Error loading Tamil:", e)
            return False

    def build_unified_index(self):
        audio_list = []
        text_list = []
        meta = []
        if self.english_data:
            audio_list.append(self.english_data['audio_embeddings'])
            text_list.append(self.english_data['text_embeddings'])
            meta.extend(self.english_data['metadata'])
        if self.tamil_data:
            audio_list.append(self.tamil_data['audio_embeddings'])
            text_list.append(self.tamil_data['text_embeddings'])
            meta.extend(self.tamil_data['metadata'])

        if not audio_list:
            raise RuntimeError("No audio embeddings loaded.")

        combined_audio_emb = np.vstack(audio_list).astype("float32")
        combined_text_emb = np.vstack(text_list).astype("float32")

        # Normalize (cosine similarity via inner product)
        faiss.normalize_L2(combined_audio_emb)
        faiss.normalize_L2(combined_text_emb)

        d_audio = combined_audio_emb.shape[1]
        d_text = combined_text_emb.shape[1]

        self.combined_audio_index = faiss.IndexFlatIP(d_audio)
        self.combined_audio_index.add(combined_audio_emb)

        self.combined_text_index = faiss.IndexFlatIP(d_text)
        self.combined_text_index.add(combined_text_emb)

        self.combined_metadata = meta

    # see earlier message for TF-IDF fallback implementation (keep as-is)
    def _text_search_fallback(self, query: str, top_k: int, language_filter: Optional[str] = None) -> List[Dict]:
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.metrics.pairwise import cosine_similarity
        # identical to your fallback implementation...
        if language_filter:
            candidates = [m for m in self.combined_metadata if m['language'] == language_filter]
        else:
            candidates = self.combined_metadata
        if not candidates:
            return []
        texts = [(m.get('transcript') or m.get('text', '')).lower() for m in candidates]
        vectorizer = TfidfVectorizer(max_features=1000)
        try:
            all_texts = texts + [query.lower()]
            tfidf_matrix = vectorizer.fit_transform(all_texts)
            query_vec = tfidf_matrix[-1]
            corpus_vecs = tfidf_matrix[:-1]
            similarities = cosine_similarity(query_vec, corpus_vecs).flatten()
            top_idx = np.argsort(similarities)[::-1][:top_k]
            results = []
            for i in top_idx:
                r = candidates[i].copy()
                r['similarity_score'] = float(similarities[i])
                results.append(r)
            return results
        except Exception as e:
            print("TF-IDF fallback error:", e)
            return [dict(m, similarity_score=0.0) for m in candidates[:top_k]]

    # search_by_text: tries CLAP encoder, falls back to TF-IDF
    def search_by_text(self, query: str, top_k: int = 10, language_filter: Optional[str] = None) -> List[Dict]:
        if self.clap_encoder is not None and self.combined_audio_index is not None:
            try:
                q_emb = self.clap_encoder.encode_texts([query])
                q_emb = np.ascontiguousarray(q_emb.astype("float32"))
                faiss.normalize_L2(q_emb)
                idx_dim = getattr(self.combined_audio_index, "d", None)
                if idx_dim is not None and q_emb.shape[1] != idx_dim:
                    raise RuntimeError(f"Dim mismatch: query {q_emb.shape[1]} vs index {idx_dim}")
                scores, indices = self.combined_audio_index.search(q_emb, top_k)
                results = []
                for s, idx in zip(scores[0], indices[0]):
                    if idx < len(self.combined_metadata):
                        r = self.combined_metadata[idx].copy()
                        r['similarity_score'] = float(s)
                        if language_filter is None or r.get('language') == language_filter:
                            results.append(r)
                return results
            except Exception as e:
                print("CLAP search failed, falling back to TF-IDF:", e)
                return self._text_search_fallback(query, top_k, language_filter)
        else:
            return self._text_search_fallback(query, top_k, language_filter)


    def search_similar_audio(self, reference_idx: int, top_k: int = 10,
                            language_filter: Optional[str] = None) -> List[Dict]:
        """
        Find similar audio by providing a reference index
        
        Args:
            reference_idx: Index of reference audio
            top_k: Number of results
            language_filter: Filter by language
            
        Returns:
            List of similar audio results
        """
        
        if reference_idx >= len(self.combined_metadata):
            print(f"❌ Invalid reference index: {reference_idx}")
            return []
        
        ref_meta = self.combined_metadata[reference_idx]
        print(f"\nFinding audio similar to:")
        print(f"  Language: {ref_meta['language']}")
        print(f"  ID: {ref_meta.get('file_id') or ref_meta.get('id')}")
        
        # Get embedding for reference audio
        ref_emb = self.combined_audio_index.reconstruct(reference_idx).reshape(1, -1)
        
        # Search
        search_k = top_k * 5 if language_filter else top_k + 1  # +1 to exclude self
        scores, indices = self.combined_audio_index.search(ref_emb, search_k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            # Skip the reference itself
            if idx == reference_idx:
                continue
                
            if idx < len(self.combined_metadata):
                result = self.combined_metadata[idx].copy()
                result['similarity_score'] = float(score)
                
                # Apply language filter
                if language_filter is None or result['language'] == language_filter:
                    results.append(result)
                    
                if len(results) >= top_k:
                    break
        
        return results
    
    def cross_lingual_search(self, query_language: str, target_language: str, 
                            query_idx: int, top_k: int = 10) -> List[Dict]:
        """
        Cross-lingual search: Find Tamil audio from English query or vice versa
        
        Args:
            query_language: 'english' or 'tamil'
            target_language: 'english' or 'tamil'
            query_idx: Index of query audio in its language dataset
            top_k: Number of results
        """
        
        if query_language == target_language:
            print("⚠️  Use search_similar_audio for same-language search")
            return self.search_similar_audio(query_idx, top_k, target_language)
        
        # Find actual index in combined dataset
        actual_idx = query_idx
        if query_language == 'tamil' and self.english_data:
            actual_idx += len(self.english_data['metadata'])
        
        if actual_idx >= len(self.combined_metadata):
            print(f"❌ Invalid query index")
            return []
        
        print(f"\nCross-lingual search:")
        print(f"  Query: {query_language}")
        print(f"  Target: {target_language}")
        
        return self.search_similar_audio(actual_idx, top_k, target_language)
    
    def get_statistics(self) -> Dict:
        """Get system statistics"""
        
        stats = {
            'total_entries': len(self.combined_metadata),
            'english_entries': sum(1 for m in self.combined_metadata if m['language'] == 'english'),
            'tamil_entries': sum(1 for m in self.combined_metadata if m['language'] == 'tamil'),
            'audio_embedding_dim': self.combined_audio_index.d if self.combined_audio_index else 0,
            'text_embedding_dim': self.combined_text_index.d if self.combined_text_index else 0,
        }
        
        return stats
    
    def save_index(self, output_path: str = "/kaggle/input/audio-retrieval/audio_retrieval/unified_index"):
        """Save unified index for later use"""
        
        output_path = Path(output_path)
        output_path.mkdir(parents=True, exist_ok=True)
        
        print(f"\nSaving unified index to {output_path}/")
        
        # Save FAISS indices
        faiss.write_index(self.combined_audio_index, str(output_path / "audio_index.faiss"))
        faiss.write_index(self.combined_text_index, str(output_path / "text_index.faiss"))
        
        # Save metadata
        with open(output_path / "metadata.json", 'w', encoding='utf-8') as f:
            json.dump(self.combined_metadata, f, indent=2, ensure_ascii=False)
        
        # Save statistics
        stats = self.get_statistics()
        with open(output_path / "statistics.json", 'w', encoding='utf-8') as f:
            json.dump(stats, f, indent=2)
        
        print("✓ Index saved successfully")
    
    def load_index(self, index_path: str = "/kaggle/input/audio-retrieval/audio_retrieval/unified_index"):
        """Load pre-built unified index"""
        
        index_path = Path(index_path)
        
        print(f"\nLoading unified index from {index_path}/")
        
        # Load FAISS indices
        self.combined_audio_index = faiss.read_index(str(index_path / "audio_index.faiss"))
        self.combined_text_index = faiss.read_index(str(index_path / "text_index.faiss"))
        
        # Load metadata
        with open(index_path / "metadata.json", 'r', encoding='utf-8') as f:
            self.combined_metadata = json.load(f)
        
        print("✓ Index loaded successfully")
        print(f"  Total entries: {len(self.combined_metadata)}")


def main():
    """Example usage"""
    
    print("\n" + "="*70)
    print("MULTILINGUAL AUDIO RETRIEVAL SYSTEM")
    print("English (LibriSpeech) + Tamil (Indic TTS)")
    print("="*70)
    
    # Initialize system
    retrieval = MultilingualAudioRetrieval()
    
    # Load English data
    english_loaded = retrieval.load_english_data(
        audio_emb_path="/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/audio_features/clap_embeddings.npy",
        text_emb_path="/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/text_features/clap_text_embeddings.npy",
        metadata_path="/kaggle/input/audio-retrieval/audio_retrieval/preprocessed_features/metadata/dataset_metadata.json"
    )
    
    # Load Tamil data
    tamil_loaded = retrieval.load_tamil_data(
        audio_emb_path="/kaggle/input/audio-retrieval/audio_retrieval/tamil_features/CLAP_audio_embeddings.npy",
        text_emb_path="/kaggle/input/audio-retrieval/audio_retrieval/tamil_features/clap_text_embeddings.npy",
        metadata_path="/kaggle/input/audio-retrieval/audio_retrieval/tamil_features/metadata.json"
    )
    
    if not (english_loaded or tamil_loaded):
        print("\n❌ Failed to load any data. Please check file paths.")
        return
    
    # Build unified index
    retrieval.build_unified_index()
    
    # Save index
    retrieval.save_index("unified_index")
    
    # Display statistics
    print("\n" + "="*70)
    print("SYSTEM STATISTICS")
    print("="*70)
    stats = retrieval.get_statistics()
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    # Example searches
    print("\n" + "="*70)
    print("EXAMPLE SEARCHES")
    print("="*70)
    
    # 1. Search in English only
    print("\n1. Searching English audio only:")
    results = retrieval.search_by_text("speech", top_k=3, language_filter='english')
    for i, result in enumerate(results, 1):
        print(f"  [{i}] {result.get('file_id', result.get('id'))} - Score: {result['similarity_score']:.4f}")
        print(f"      Text: {result.get('transcript', result.get('text', ''))[:60]}...")
    
    # 2. Search in Tamil only
    print("\n2. Searching Tamil audio only:")
    results = retrieval.search_by_text("தமிழ்", top_k=3, language_filter='tamil')
    for i, result in enumerate(results, 1):
        print(f"  [{i}] {result.get('id')} - Score: {result['similarity_score']:.4f}")
        print(f"      Text: {result.get('text', '')[:60]}...")
    
    # 3. Find similar audio (English)
    if english_loaded:
        print("\n3. Finding similar English audio (reference index 0):")
        results = retrieval.search_similar_audio(0, top_k=3, language_filter='english')
        for i, result in enumerate(results, 1):
            print(f"  [{i}] {result.get('file_id')} - Score: {result['similarity_score']:.4f}")
    
    # 4. Cross-lingual search
    if english_loaded and tamil_loaded:
        print("\n4. Cross-lingual search (English → Tamil):")
        results = retrieval.cross_lingual_search('english', 'tamil', 0, top_k=3)
        for i, result in enumerate(results, 1):
            print(f"  [{i}] {result.get('id')} - Score: {result['similarity_score']:.4f}")
    
    print("\n" + "="*70)
    print("✅ SYSTEM READY FOR QUERIES")
    print("="*70)


if __name__ == "__main__":
    main()



MULTILINGUAL AUDIO RETRIEVAL SYSTEM
English (LibriSpeech) + Tamil (Indic TTS)

Saving unified index to unified_index/
✓ Index saved successfully

SYSTEM STATISTICS
  total_entries: 8099
  english_entries: 2703
  tamil_entries: 5396
  audio_embedding_dim: 512
  text_embedding_dim: 512

EXAMPLE SEARCHES

1. Searching English audio only:
  [1] 6313-66129-0009 - Score: 0.0000
      Text: PRESIDENT BROWN I WITHDRAW MY CRITICISM I OFFER YOU MY HUMBL...
  [2] 1673-143397-0005 - Score: 0.0000
      Text: A RUMOR WAS SPREAD AMONG THE CHRISTIANS THAT THE DAUGHTER OF...
  [3] 174-50561-0008 - Score: 0.0000
      Text: BUT IF I PLAY YOU A ROUNDEL LADY GET ME A GIFT FROM THE EMPE...

2. Searching Tamil audio only:
  [1] train_tamilfem_03013 - Score: 0.7101
      Text: இன்னிக்கு தமிழ்நாட்டுலே தமிழ் மொத்தமா அழிஞ்சு போச்சுனு எழுதி...
  [2] train_tamilfem_03125 - Score: 0.6383
      Text: தமிழ் மென்பொருள் தமிழ் எழுத்துக்கள் மற்றும் தமிழ்க் கணிப்பொற...
  [3] train_tamilfem_02640 - Score: 0.5963
      T

In [7]:
"""
Interactive Query Interface for Multilingual Audio Retrieval
"""

import sys
from pathlib import Path
import numpy as np
import json

# Import the retrieval system
# from multilingual_retrieval_system import MultilingualAudioRetrieval


class InteractiveRetrieval:
    """Interactive interface for querying the retrieval system"""
    
    def __init__(self, retrieval_system):
        self.system = retrieval_system
    
    def display_menu(self):
        """Display main menu"""
        print("\n" + "="*70)
        print("MULTILINGUAL AUDIO RETRIEVAL - INTERACTIVE MODE")
        print("="*70)
        print("\nOptions:")
        print("  1. Search by text (all languages)")
        print("  2. Search by text (English only)")
        print("  3. Search by text (Tamil only)")
        print("  7. View system statistics")
        print("  8. Browse random samples")
        #print("  9. Export search results to CSV")
        print("  0. Exit")
        print("="*70)
    
    def search_text_all(self):
        """Search by text across all languages"""
        query = input("\nEnter search query (English or Tamil): ").strip()
        if not query:
            print("❌ Empty query")
            return
        
        top_k = int(input("Number of results (default 10): ") or "10")
        
        results = self.system.search_by_text(query, top_k=top_k)
        self.display_results(results)
    
    def search_text_english(self):
        """Search English audio only"""
        query = input("\nEnter search query (English): ").strip()
        if not query:
            print("❌ Empty query")
            return
        
        top_k = int(input("Number of results (default 10): ") or "10")
        
        results = self.system.search_by_text(query, top_k=top_k, language_filter='english')
        self.display_results(results)
    
    def search_text_tamil(self):
        """Search Tamil audio only"""
        query = input("\nEnter search query (Tamil): ").strip()
        if not query:
            print("❌ Empty query")
            return
        
        top_k = int(input("Number of results (default 10): ") or "10")
        
        results = self.system.search_by_text(query, top_k=top_k, language_filter='tamil')
        self.display_results(results)
    
    def search_similar_audio(self):
        """Find similar audio by index"""
        print(f"\nTotal entries: {len(self.system.combined_metadata)}")
        
        idx = int(input("Enter reference audio index: "))
        if idx < 0 or idx >= len(self.system.combined_metadata):
            print(f"❌ Invalid index. Must be 0-{len(self.system.combined_metadata)-1}")
            return
        
        # Show reference audio info
        ref = self.system.combined_metadata[idx]
        print(f"\nReference audio:")
        print(f"  Language: {ref['language']}")
        print(f"  ID: {ref.get('file_id') or ref.get('id')}")
        print(f"  Text: {ref.get('transcript') or ref.get('text', '')[:80]}...")
        
        lang_filter = input("\nFilter by language? (english/tamil/all) [all]: ").strip().lower()
        if lang_filter not in ['english', 'tamil', 'all']:
            lang_filter = None
        elif lang_filter == 'all':
            lang_filter = None
        
        top_k = int(input("Number of results (default 10): ") or "10")
        
        results = self.system.search_similar_audio(idx, top_k=top_k, language_filter=lang_filter)
        self.display_results(results)
    
    def cross_lingual_en_to_ta(self):
        """Cross-lingual: English to Tamil"""
        if not self.system.english_data or not self.system.tamil_data:
            print("❌ Both English and Tamil data required")
            return
        
        max_idx = len(self.system.english_data['metadata']) - 1
        print(f"\nEnglish audio indices: 0-{max_idx}")
        
        idx = int(input("Enter English audio index: "))
        if idx < 0 or idx > max_idx:
            print(f"❌ Invalid index")
            return
        
        # Show query audio
        ref = self.system.english_data['metadata'][idx]
        print(f"\nQuery (English):")
        print(f"  ID: {ref['file_id']}")
        print(f"  Text: {ref.get('transcript', '')[:80]}...")
        
        top_k = int(input("Number of Tamil results (default 10): ") or "10")
        
        results = self.system.cross_lingual_search('english', 'tamil', idx, top_k=top_k)
        self.display_results(results, title="Similar Tamil Audio")
    
    def cross_lingual_ta_to_en(self):
        """Cross-lingual: Tamil to English"""
        if not self.system.english_data or not self.system.tamil_data:
            print("❌ Both English and Tamil data required")
            return
        
        max_idx = len(self.system.tamil_data['metadata']) - 1
        print(f"\nTamil audio indices: 0-{max_idx}")
        
        idx = int(input("Enter Tamil audio index: "))
        if idx < 0 or idx > max_idx:
            print(f"❌ Invalid index")
            return
        
        # Show query audio
        ref = self.system.tamil_data['metadata'][idx]
        print(f"\nQuery (Tamil):")
        print(f"  ID: {ref['id']}")
        print(f"  Text: {ref.get('text', '')[:80]}...")
        
        top_k = int(input("Number of English results (default 10): ") or "10")
        
        results = self.system.cross_lingual_search('tamil', 'english', idx, top_k=top_k)
        self.display_results(results, title="Similar English Audio")
    
    def view_statistics(self):
        """Display system statistics"""
        stats = self.system.get_statistics()
        
        print("\n" + "="*70)
        print("SYSTEM STATISTICS")
        print("="*70)
        
        for key, value in stats.items():
            print(f"  {key.replace('_', ' ').title()}: {value}")
        
        print("\n" + "="*70)
    
    def browse_samples(self):
        """Browse random samples"""
        import random
        
        num_samples = int(input("\nNumber of samples to show (default 5): ") or "5")
        lang = input("Language (english/tamil/all) [all]: ").strip().lower()
        
        # Filter by language
        if lang == 'english':
            candidates = [i for i, m in enumerate(self.system.combined_metadata) 
                         if m['language'] == 'english']
        elif lang == 'tamil':
            candidates = [i for i, m in enumerate(self.system.combined_metadata) 
                         if m['language'] == 'tamil']
        else:
            candidates = list(range(len(self.system.combined_metadata)))
        
        # Random sample
        sample_indices = random.sample(candidates, min(num_samples, len(candidates)))
        
        print("\n" + "="*70)
        print("RANDOM SAMPLES")
        print("="*70)
        
        for i, idx in enumerate(sample_indices, 1):
            item = self.system.combined_metadata[idx]
            print(f"\n[{i}] Index: {idx}")
            print(f"    Language: {item['language']}")
            print(f"    ID: {item.get('file_id') or item.get('id')}")
            print(f"    Text: {item.get('transcript') or item.get('text', '')[:80]}...")
            if 'path' in item:
                print(f"    Path: {item['path']}")
    
    def export_results(self):
        """Export search results to CSV"""
        import pandas as pd
        
        query = input("\nEnter search query: ").strip()
        if not query:
            print("❌ Empty query")
            return
        
        top_k = int(input("Number of results (default 100): ") or "100")
        output_file = input("Output filename [search_results.csv]: ").strip() or "search_results.csv"
        
        results = self.system.search_by_text(query, top_k=top_k)
        
        if not results:
            print("❌ No results found")
            return
        
        df = pd.DataFrame(results)
        df.to_csv(output_file, index=False, encoding='utf-8')
        
        print(f"✓ Exported {len(results)} results to {output_file}")
    
    def display_results(self, results, title="Search Results"):
        """Display search results"""
        if not results:
            print("\n❌ No results found")
            return
        
        print("\n" + "="*70)
        print(title.upper())
        print("="*70)
        
        for i, result in enumerate(results, 1):
            print(f"\n[{i}] Score: {result['similarity_score']:.4f}")
            print(f"    Language: {result['language']}")
            print(f"    ID: {result.get('file_id') or result.get('id')}")
            
            text = result.get('transcript') or result.get('text', '')
            if text:
                print(f"    Text: {text[:100]}{'...' if len(text) > 100 else ''}")
            
            if 'path' in result:
                print(f"    Path: {result['path']}")
            elif 'audio_file' in result:
                print(f"    File: {result['audio_file']}")
        
        print("\n" + "="*70)
    
    def run(self):
        """Run interactive loop"""
        while True:
            self.display_menu()
            
            try:
                choice = input("\nSelect option: ").strip()
                
                if choice == '1':
                    self.search_text_all()
                elif choice == '2':
                    self.search_text_english()
                elif choice == '3':
                    self.search_text_tamil()
                elif choice == '4':
                    self.search_similar_audio()
                elif choice == '5':
                    self.cross_lingual_en_to_ta()
                elif choice == '6':
                    self.cross_lingual_ta_to_en()
                elif choice == '7':
                    self.view_statistics()
                elif choice == '8':
                    self.browse_samples()
                elif choice == '9':
                    self.export_results()
                elif choice == '0':
                    print("\nExiting... Goodbye!")
                    break
                else:
                    print("❌ Invalid option")
                
                input("\nPress Enter to continue...")
                
            except KeyboardInterrupt:
                print("\n\nExiting... Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ Error: {e}")
                input("\nPress Enter to continue...")


def main():
    """Main entry point"""
    
    # Import here to avoid circular dependency
    #from multilingual_retrieval_system import MultilingualAudioRetrieval
    
    print("\n" + "="*70)
    print("INITIALIZING MULTILINGUAL AUDIO RETRIEVAL SYSTEM")
    print("="*70)
    
    # Initialize system
    retrieval = MultilingualAudioRetrieval()
    
    # Check if unified index exists
    if Path("unified_index/metadata.json").exists():
        print("\nLoading pre-built index...")
        retrieval.load_index("unified_index")
    else:
        print("\nBuilding index from scratch...")
        
        # Load English data
        retrieval.load_english_data()
        
        # Load Tamil data
        retrieval.load_tamil_data()
        
        # Build index
        retrieval.build_unified_index()
        
        # Save for next time
        retrieval.save_index("unified_index")
    
    # Start interactive interface
    interface = InteractiveRetrieval(retrieval)
    interface.run()


if __name__ == "__main__":
    main()



INITIALIZING MULTILINGUAL AUDIO RETRIEVAL SYSTEM

Loading pre-built index...

Loading unified index from unified_index/
✓ Index loaded successfully
  Total entries: 8099

MULTILINGUAL AUDIO RETRIEVAL - INTERACTIVE MODE

Options:
  1. Search by text (all languages)
  2. Search by text (English only)
  3. Search by text (Tamil only)
  7. View system statistics
  8. Browse random samples
  0. Exit



Select option:  1

Enter search query (English or Tamil):  இப்படி
Number of results (default 10):  3



Searching by text: 'இப்படி...'
Language filter: All languages

SEARCH RESULTS

[1] Score: 0.6530
    Language: tamil
    ID: train_tamilfem_03517
    Text: ஆனாலும் நான் அப்படித் தான் தயவுசெய்து என்னை இப்படியே உள்வாங்கிக் கொள்
    File: train_tamilfem_03517.wav

[2] Score: 0.6104
    Language: tamil
    ID: train_tamilfem_00005
    Text: இப்படி அசடு மாதிரி ஃப்ளைட்டை கோட்டை விட்டுட்டோம்னு வெட்கமா இருந்தது
    File: train_tamilfem_00005.wav

[3] Score: 0.6010
    Language: tamil
    ID: train_tamilfem_04911
    Text: அதனாலதான் இப்படி ஒரு பொய் சொன்னேன்னு சொன்னான்
    File: train_tamilfem_04911.wav



Exiting... Goodbye!
